In [36]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.optimize as sciop

In [ ]:
# User Defined Stock Tickers 
assetlist = ["AAPL","MSFT","GOOGL","META","SPY"]
marketlist = ["US","Japan","UK"]


In [62]:
# Using YFinance to obtain stock close Price
stock_price = yf.download(tickers=assetlist,period = "10y", interval ="1mo")
# Find stock return
stock_return = stock_price["Close"].pct_change().dropna()

# Finding Market
yf.Market("US")


/var/folders/yj/_sykjyks31j7t9pdm3jfl0v00000gn/T/ipykernel_92539/2963078414.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_price = yf.download(tickers=assetlist,period = "10y", interval ="1mo")
[*********************100%***********************]  5 of 5 completed


In [ ]:
# Get mean and stdev of the stocks
mean_return = stock_return.mean()
stdev_return = np.std(stock_return,axis=0)

# Merge into a table
asset_risk_return_df = pd.DataFrame([mean_return, stdev_return],index=["Return", "Deviation"]).T 

# Get correlation between each stock
asset_correlation_matrix_df= stock_return.corr()

In [56]:
# Finding Weightage of Portfolio on Global Minimum Variance

# Define order
asset_order = asset_risk_return_df.index
# Reindex correlation matrix rows and columns
asset_correlation_matrix_df = asset_correlation_matrix_df.loc[asset_order, asset_order]

# Get deviations
deviations = asset_risk_return_df["Deviation"].values

# Multiply element-wise
cov_matrix = np.outer(deviations, deviations) * asset_correlation_matrix_df.values

# Step 2: Create covariance matrix
# correlation_matrix = asset_correlation_matrix_df  # ensure correct order
# cov_matrix = np.outer(std_devs, std_devs) * correlation_matrix

# Step 3: Define Minimum Variance Function
def global_min_var_portfolio(weights):
    return weights.T @ cov_matrix @ weights # Matrix multiplication to find variance

# Step 4: Define Constraints and Bounds
number_of_assets = len(assetlist) # asset count
init_guess = np.ones(number_of_assets)/number_of_assets # start with equal weightages
bounds = [(0,1) for a in range(number_of_assets)] # Bound the Asset weightage
constraints = {'type': 'eq', # Equality Constraint
               'fun' : lambda w : np.sum(w)-1 # Must Equal to 0
               # This constraint ensures that sum of all weight = 1
               # or sum of weight - 1 = 0
               }

# Step 5: Solve for Efficient Portfolio
result = sciop.minimize(global_min_var_portfolio, init_guess, method='SLSQP',
                bounds=bounds, constraints=constraints)

# Step 6: Output Weightages
min_variance_weights = result.x
MVP_portfolio = dict(zip(assetlist, min_variance_weights))
print(f'''
      MVP Portfolio Generated:
      {MVP_portfolio}
      ''')


      MVP Portfolio Generated:
      {'AAPL': np.float64(0.0), 'MSFT': np.float64(0.0), 'GOOGL': np.float64(5.0756441812853116e-17), 'META': np.float64(0.22522840378108), 'SPY': np.float64(0.77477159621892)}
      
